### **1. Paramètres**

In [1]:
# Importation des bibliothèques
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from functions_plots import top_cultivars, boxnotch_lidar_metrics

In [2]:
# Chemin vers les fichiers
csv_path = '../data_final/tableaux/'
output_path = '../results/3_lidar_metrics'

# Vérifier si le dossier de sortie existe ; le créer si nécessaire
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Lire le fichier CSV
df_px_filtre_lidar = pd.read_csv(os.path.join(csv_path, 'df_pixel_filtre_lidar.csv'), dtype={'id_parc': str})

### **2. Faire les boxplots**

In [3]:
# Faire un dicitonnaires avec les cultivars principaux
dict_top_cultivars = top_cultivars(df_px_filtre_lidar, max_age=12, min_years=2, top_n=20)
print(f"Nombre de cultivars sélectionnés: {len(dict_top_cultivars)}")

# Imprimer les cultivars triés par nombre de pixels
for cultivar, count in dict_top_cultivars.items():
    print(f"{cultivar}: {count} pixels")

Nombre de cultivars sélectionnés: 16
Koster: 783 pixels
I45/51: 580 pixels
Raspalje: 536 pixels
Polargo: 516 pixels
I214: 497 pixels
Soligo: 317 pixels
Diva: 182 pixels
Tucano: 117 pixels
Moncalvo: 114 pixels
Senna: 114 pixels
San Martino: 83 pixels
Rona: 71 pixels
Aleramo: 64 pixels
Hoogvorst: 30 pixels
Delvignac: 26 pixels
Vesten: 5 pixels


In [4]:
# Prendre juste les tops cultivar  pour faire les boxplots
df_px_filtre_lidar = df_px_filtre_lidar[df_px_filtre_lidar['cultivar_n'].isin(dict_top_cultivars)]
df_px_filtre_lidar = df_px_filtre_lidar[(df_px_filtre_lidar['age_plan'] >= 1) & (df_px_filtre_lidar['age_plan'] <= 12)]
df_px_filtre_lidar['age_plan'] = df_px_filtre_lidar['age_plan'].astype(int)

In [5]:
# Obtenir toutes les valeurs uniques de la colonne 'source' dans le DataFrame filtré
unique_sources = df_px_filtre_lidar['source'].unique()

# Générer une palette de couleurs avec 'tab10' pour chaque valeur unique de 'source'
source_colors = plt.cm.tab10(np.linspace(0, 1, len(unique_sources)))
source_color_map = dict(zip(unique_sources, source_colors))

In [6]:
metrics = ['grid_CC', 'grid_PAI', 'grid_ENL', 'grid_VCI', 'grid_MOCH']

# Calculer les limites Y pour chaque métrique
y_limits = {}
for metric in metrics:
    y_min = df_px_filtre_lidar[metric].min()
    y_max = df_px_filtre_lidar[metric].max()
    y_limits[metric] = (y_min, y_max)

In [7]:
# Générer les graphiques
pdf_path = os.path.join(output_path, "Boxnotch_lidar_metrics.pdf")
with PdfPages(pdf_path) as pdf:
    for cultivar_index, (cultivar, _) in enumerate(dict_top_cultivars.items(), start=1):
        for metric in metrics:
            boxnotch_lidar_metrics(
                df=df_px_filtre_lidar,
                cultivar=cultivar,
                metric=metric,
                color_map=source_color_map,
                y_limits=y_limits[metric],
                pdf=pdf,
                output_path=output_path,
                cultivar_index=cultivar_index
            )

print(f"PDF avec tous les graphiques sauvegardé à: {pdf_path}")


PDF avec tous les graphiques sauvegardé à: ../results/3_lidar_metrics\Boxnotch_lidar_metrics.pdf
